<a href="https://colab.research.google.com/github/dpcks/playdata/blob/main/data_processing/(pandas_%EC%9D%B4%EC%9A%A9%ED%95%B4%EC%84%9C_feature_%EB%A7%8C%EB%93%A4%EA%B8%B0)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# 다음의 데이터 경로는 본인 경로에 맞게 수정하시오.

In [ ]:
DATA_PATH = "/content/drive/MyDrive/data/"

In [ ]:
df = pd.read_csv(f"{DATA_PATH}trans_train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167465 entries, 0 to 167464
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   고객번호    167465 non-null  int64 
 1   구매일자    167465 non-null  object
 2   지점      167465 non-null  object
 3   상품번호    167465 non-null  int64 
 4   중분류     167465 non-null  object
 5   대분류     167465 non-null  object
 6   구매가격    167465 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.9+ MB


In [ ]:
df.head()

,고객번호,구매일자,지점,상품번호,중분류,대분류,구매가격
0,10004,2007-06-26 00:00:00,강남점,234160,영 트렌디,캐주얼,2720000
1,10004,2007-07-08 00:00:00,본 점,165162,4대 B/D,피혁잡화,8550000
2,10004,2007-07-14 00:00:00,본 점,663110,국산냉장고.세탁기,대형가전,28800000
3,10004,2007-07-28 00:00:00,본 점,73119,상품군미지정,기타,81500
4,10004,2007-07-28 00:00:00,본 점,73119,주류,주류,2037500


- datetime 형식으로 변환

In [ ]:
df["구매일자"] = pd.to_datetime(df["구매일자"])

- 날짜끼리 연산이 가능하다.
- 날짜끼리 연산시 Timedelta 타입이 반환된다.
- Timedelta 타입은 Datetime 의 하위 객체로 두개 날짜나 시간의 차이인 기간을 나타낸다.

In [ ]:
time_delta = df["구매일자"].max() - df["구매일자"].min()
time_delta

Timedelta('363 days 00:00:00')

- 일수 뽑기

In [ ]:
time_delta.days

363

- 초 뽑기

In [ ]:
time_delta.total_seconds()

31363200.0

# Feature Engineering
 - 행은 그대로 유지
 

- 새로 만든 feature 와 병합할 고객 번호로만 이루어진 데이터프레임을 만든다.

In [ ]:
features = pd.DataFrame(df["고객번호"].unique(),columns=["고객번호"])
features.head()

,고객번호
0,10004
1,10013
2,10023
3,10040
4,10052


## 구매일자를 이용한 특성생성

- 내점일수
- 구매주기
- 주말방문비율
- 주구매요일
- 봄구매비율
- 여름구매비율
- 가을구매비율
- 겨울구매비율
- 일평균구매건수
- 거래개월수

In [ ]:
df["구매일자"].dt.weekday > 4 # 0이 월요일 

0         False
1          True
2          True
3          True
4          True
          ...  
167460    False
167461     True
167462    False
167463    False
167464    False
Name: 구매일자, Length: 167465, dtype: bool

In [ ]:
(df["구매일자"].dt.weekday > 4).mean()

0.29615143462813126

In [ ]:
agg_list = [
    # (컬럼명, 집계방식)
    ("내점일수","nunique"),
    ("구매주기",lambda x : (x.max() - x.min()).days / x.nunique() ), # 최근 구매일과 최초 구매일의 차이를 구하고 내점일수로 나누면 구매주기라고 볼수있음
    ("주말방문비율", lambda x : (x.dt.weekday > 4).mean() ), # 4보다 크면 토,일 이니까!!
    ("주구매요일" , lambda x: x.dt.weekday.mode()[0] ), # 최빈값이 여러개 일수도 있어서..
    ("봄구매비율" , lambda x : (x.dt.month.isin([3,4,5])).mean() ),
    ("여름구매비율" , lambda x : (x.dt.month.isin([6,7,8])).mean() ),
    ("가을구매비율" , lambda x : (x.dt.month.isin([9,10,11])).mean() ),
    ("겨울구매비율" , lambda x : (x.dt.month.isin([12,1,2])).mean() ),
    ("일평균구매건수" , lambda x : x.count() / x.nunique() ), # 구매건수를 구매일자로 나누면 일평균 구매건수라고 볼수있음.
    ("거래개월수" , lambda x : x.dt.month.nunique() )

]

tmp = df.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left",on = "고객번호") # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9
...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,12
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,11
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,2
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,6


## 지점을 이용한 특성생성
- 방문지점수
- 주구매지점

In [ ]:
agg_list = [
    # (컬럼명, 집계방식)
    ("방문지점수","nunique"),
    ("주구매지점" , lambda x : x.mode()[0])

]

tmp = df.groupby("고객번호")["지점"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left",on = "고객번호") # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,12,1,부산본점
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,11,2,강남점
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,2,1,본 점
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,6,1,본 점


## 상품번호를 이용한 특성생성
- 상품번호수

In [ ]:
agg_list = [
             ('상품번호수', 'nunique'),
             ]

tmp = df.groupby('고객번호')["상품번호"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left') # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점,상품번호수
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점,45
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점,10
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점,30
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점,34
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점,57


## 중분류를 이용한 특성생성
- 중분류수
- 주구매중분류

In [ ]:
agg_list = [
            ('중분류수', 'nunique'),
            ('주구매중분류', lambda x: x.mode()[0]), 
            ]

tmp = df.groupby('고객번호')["중분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점,상품번호수,중분류수,주구매중분류
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점,45,39,상품군미지정
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점,10,12,N.B정장
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점,30,23,N/B골프의류
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점,34,29,어덜트
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점,57,37,시티웨어


## 대분류를 이용한 특성생성
- 대분류수
- 주구매대분류

In [ ]:
agg_list = [
            ('대분류수', 'nunique'),
            ('주구매대분류', lambda x: x.mode()[0]),
            ]

tmp = df.groupby('고객번호')["대분류"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,거래개월수,방문지점수,주구매지점,상품번호수,중분류수,주구매중분류,대분류수,주구매대분류
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,9,3,본 점,45,39,상품군미지정,25,가공식품
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,5,3,본 점,10,12,N.B정장,9,가공식품
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,11,3,잠실점,30,23,N/B골프의류,17,골프
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,10,2,부산본점,34,29,어덜트,15,명품
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,9,4,인천점,57,37,시티웨어,20,시티웨어


## 구매가격을 이용한 특성생성
- 총구매액
- 구매건수
- 평균구매액
- 최대구매액
- 최소구매액
- 환불금액
- 환불건수
- 구매금액표준편차

In [ ]:
agg_list = [
        ('총구매액','sum'),
        ('구매건수', 'count'), 
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ), # seires의 마스킹 bool 형태로 나오기떄문 true들을 sum한 값
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ]

tmp = df.groupby('고객번호')["구매가격"].agg(agg_list).reset_index()

features = features.merge(tmp, how='left')  # features 변수에 merge 한 결과를 확인하고 변수에 넣으세요!!

features.head()

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,25,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,9,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,17,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,15,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,20,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06


In [ ]:
df["구매가격"].std()

6556450.427256041

In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2553
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   고객번호      2554 non-null   int64  
 1   내점일수      2554 non-null   int64  
 2   구매주기      2554 non-null   float64
 3   주말방문비율    2554 non-null   float64
 4   주구매요일     2554 non-null   int64  
 5   봄구매비율     2554 non-null   float64
 6   여름구매비율    2554 non-null   float64
 7   가을구매비율    2554 non-null   float64
 8   겨울구매비율    2554 non-null   float64
 9   일평균구매건수   2554 non-null   float64
 10  거래개월수     2554 non-null   int64  
 11  방문지점수     2554 non-null   int64  
 12  주구매지점     2554 non-null   object 
 13  상품번호수     2554 non-null   int64  
 14  중분류수      2554 non-null   int64  
 15  주구매중분류    2554 non-null   object 
 16  대분류수      2554 non-null   int64  
 17  주구매대분류    2554 non-null   object 
 18  총구매액      2554 non-null   int64  
 19  구매건수      2554 non-null   int64  
 20  평균구매액     2554 non-null   floa

- 특성(feature) 생성 과정에서 결측치가 생기는 부분은 신경 쓰지말자
    - 나중에 머신러닝모델에 넣기전에 채우면 된다.
- 범주형 특성을 추가하는 과정에서 인코딩을 어떻게 해야할지도 신경 쓰지말자
    - 나중에 머신러닝모델에 넣기전에 고민해도 된다.
- 결론은 새로운 특성을 추가하는데 집중하자는 말입니다. 우선 특성추가하는데 집중하세요~

# 여기서 부터 features 변수에 특성을 추가해주면 됩니다.
- features 변수에 들어있는 데이터의 행이 줄어들거나 늘어나서는 안됩니다!! 
- 항상 merge 한 결과를 확인하고 features 변수에 할당해주세요.
- 추가한 특성에 대한 간단한 설명 넣어주면 감사!

In [ ]:
features = features.iloc[:,:26]
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2553
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   고객번호      2554 non-null   int64  
 1   내점일수      2554 non-null   int64  
 2   구매주기      2554 non-null   float64
 3   주말방문비율    2554 non-null   float64
 4   주구매요일     2554 non-null   int64  
 5   봄구매비율     2554 non-null   float64
 6   여름구매비율    2554 non-null   float64
 7   가을구매비율    2554 non-null   float64
 8   겨울구매비율    2554 non-null   float64
 9   일평균구매건수   2554 non-null   float64
 10  거래개월수     2554 non-null   int64  
 11  방문지점수     2554 non-null   int64  
 12  주구매지점     2554 non-null   object 
 13  상품번호수     2554 non-null   int64  
 14  중분류수      2554 non-null   int64  
 15  주구매중분류    2554 non-null   object 
 16  대분류수      2554 non-null   int64  
 17  주구매대분류    2554 non-null   object 
 18  총구매액      2554 non-null   int64  
 19  구매건수      2554 non-null   int64  
 20  평균구매액     2554 non-null   floa

# 상품번호를 통한 분류


In [ ]:
agg_list=[
    ("많이사간 상품번호",lambda x : x.mode()[0]) # 각 고개들마다 많이 산 상품을 통해서 고객의 특성 파악
]
tmp = df.groupby("고객번호")["상품번호"].agg(agg_list).reset_index()
features = features.merge(tmp, how="left")
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,많이사간 상품번호
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,85198
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,85186
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,143101
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,740177
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,85115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,85138
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,37140
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,300113
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,37155


In [ ]:
features = features.iloc[:,:27]
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,많이사간 상품번호
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,85198
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,85186
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,143101
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,740177
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,85115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,85138
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,37140
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,300113
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,37155


# 대분류를 이용한 특성 생성
 
- 🔧 시대상 2007년이라는 것을 고려해서 남성들이 자주 사용할것같은 품목과 여성들이 자주 사용할것 같은 품목들을 분류했다.
- 🔧 식료품 구매비율은 주부들이 샀을거라고 추정해 여성이라고 생각하면서 분류
- 🔧 가전/가구/전자제품은 주로 부부들이 같이 생각하고 정할것 같아서 따로 분류했다.
- 🔧 자식용품구매비율은 시대상 남성들은 회사를 가고 어머니들이 자식들과 같이가서 자식들의 용품을 구매할 가능성이 높아서 여성이라고 생각해 분류


In [ ]:
agg_list= [
    ("남성의류구매비율", lambda x: (x.str.contains("남성|시티웨어|셔츠").mean())), # 시티웨어를 포함한 이유는 시티웨어가 직장인이 주로 입는 옷인데 시대상 남자들이 직장을 더 많이 다녔을꺼라고 생각
    ("스포츠관련구매비율", lambda x: (x.str.contains("골프|스포츠").mean())),
    ("여성용품구매비율", lambda x :(x.str.contains("란제리|화장품|보석|구두|액세서리|웨딩|모피|차|명품").mean())), # 시대상 차종류나 커피는 여자들이 더 많이 마심 남자들은 주로 술
    ("남성의류제외구매비율", lambda x: (x.str.contains("캐주얼|디자이너|기타의류").mean())),
    ("식료품구매 비율", lambda x:(x.str.contains("식품|수산|축산|육류|반찬").mean())),
    ("가전,가구구매 비율", lambda x:(x.str.contains("가전|가구|침구").mean())), # 가전제품이나 가구들은 주로 신혼들이나 부부들이 같이 올거같아서 따로 분류
    ("전자제품구매 비율", lambda x:(x.str.contains("통신|전기").mean())),
    ("잡화구매 비율", lambda x:(x.str.contains("잡화").mean())),
    ("자식용품구매 비율", lambda x:(x.str.contains("아동|교복|문구")).mean())
    ]
tmp = df.groupby("고객번호")["대분류"].agg(agg_list).reset_index()
features = features.merge(tmp, how="left")
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,많이사간 상품번호,남성의류구매비율,스포츠관련구매비율,여성용품구매비율,남성의류제외구매비율,식료품구매 비율,"가전,가구구매 비율",전자제품구매 비율,잡화구매 비율,자식용품구매 비율
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,85198,0.088235,0.073529,0.117647,0.088235,0.191176,0.073529,0.0,0.132353,0.000000
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,85186,0.235294,0.058824,0.058824,0.000000,0.411765,0.058824,0.0,0.000000,0.000000
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,143101,0.069767,0.325581,0.069767,0.093023,0.255814,0.000000,0.0,0.023256,0.000000
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,740177,0.015152,0.000000,0.454545,0.045455,0.121212,0.000000,0.0,0.015152,0.106061
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,85115,0.243243,0.027027,0.081081,0.108108,0.198198,0.018018,0.0,0.027027,0.126126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,85138,0.005391,0.000000,0.040431,0.021563,0.584906,0.008086,0.0,0.080863,0.002695
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,37140,0.016529,0.008264,0.066116,0.016529,0.239669,0.033058,0.0,0.074380,0.000000
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,300113,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,37155,0.181818,0.000000,0.181818,0.090909,0.090909,0.000000,0.0,0.000000,0.000000


## 수치가 맞는지 검산

In [ ]:
mask = df["고객번호"] == 10004
df2 = df.loc[mask]

df2["대분류"].str.contains("남성").mean() + df2["대분류"].str.contains("시티웨어").mean() 
# ("남성의류비율", lambda x: (x.str.contains("남성|시티웨어").mean())) 맞게 작동.

0.07352941176470588

In [ ]:
agg_list=[
    ("남성포함대분류비", lambda x: (x.str.contains("남성").mean())),
    ("시티웨어포함대분류비", lambda x: (x.str.contains("시티웨어").mean())),
    ("셔츠포함대분류비", lambda x: (x.str.contains("셔츠").mean())),
    ("합계", lambda x: (x.str.contains("남성|시티웨어|셔츠").mean()))
]
df.groupby("고객번호")["대분류"].agg(agg_list).reset_index()

,고객번호,남성포함대분류비,시티웨어포함대분류비,셔츠포함대분류비,합계
0,10004,0.073529,0.000000,0.014706,0.088235
1,10013,0.176471,0.000000,0.058824,0.235294
2,10023,0.000000,0.023256,0.046512,0.069767
3,10040,0.015152,0.000000,0.000000,0.015152
4,10052,0.000000,0.234234,0.009009,0.243243
...,...,...,...,...,...
2549,49813,0.000000,0.000000,0.005391,0.005391
2550,49892,0.000000,0.016529,0.000000,0.016529
2551,49925,0.000000,0.000000,0.000000,0.000000
2552,49943,0.090909,0.090909,0.000000,0.181818


#중분류를 이용한 특성 추가
🔧 중분류에는 대분류보다 조금더 상세한 품목들이 나와있어서 좀더 명확한 남자와 여자의 용품들을 선별해서 비율로 나타냈다.


In [ ]:
agg_list = [
    ("상세한 남자 용품 비율",lambda x:(x.str.contains("정장|넥타이|남성|골프|스포츠|시티|셔츠|신사|시계|남화")).mean()),
    # 시계는 여자도 사는 경우도 있지만 시대상 주로 남자들이 회사에서 많이 찰거라고 생각해서 남성 용품으로 분류
    ("상세한 여자 용품 비율",lambda x:(x.str.contains("보석|여화|스카프|숙녀|화장품|핸드백|뷰티|란제리|엘리든|머플러")).mean()),# 엘리든은 여성수입의류이다
]
tmp = df.groupby("고객번호")["중분류"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left")
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,스포츠관련구매비율,여성용품구매비율,남성의류제외구매비율,식료품구매 비율,"가전,가구구매 비율",전자제품구매 비율,잡화구매 비율,자식용품구매 비율,상세한 남자 용품 비율,상세한 여자 용품 비율
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,0.073529,0.117647,0.088235,0.191176,0.073529,0.0,0.132353,0.000000,0.117647,0.058824
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,0.058824,0.058824,0.000000,0.411765,0.058824,0.0,0.000000,0.000000,0.176471,0.058824
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,0.325581,0.069767,0.093023,0.255814,0.000000,0.0,0.023256,0.000000,0.348837,0.046512
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,0.000000,0.454545,0.045455,0.121212,0.000000,0.0,0.015152,0.106061,0.015152,0.106061
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,0.027027,0.081081,0.108108,0.198198,0.018018,0.0,0.027027,0.126126,0.189189,0.081081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,0.000000,0.040431,0.021563,0.584906,0.008086,0.0,0.080863,0.002695,0.002695,0.040431
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,0.008264,0.066116,0.016529,0.239669,0.033058,0.0,0.074380,0.000000,0.024793,0.033058
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.333333
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,0.000000,0.181818,0.090909,0.090909,0.000000,0.0,0.000000,0.000000,0.181818,0.181818


# 지점을 이용해서 특성 추가
- 🔧 수도권과 지방을 따로 분류해서 한 지역만 가지않고 두지역 이상 갔을 경우 운송업자라고 가정했다 대부분의 운송업자는 남자이기때문에 운송업자일 경우 남자
혹, 운송업자가 아니라고해도 대부분 남성들이 출장을 가서 백화점을 들릴경우를 가정해서 남자라고 생각했다.

    - ⚡ 여기서 두지역 이상으로 간다고 했을경우 운송업자일수도 있지만 명절날 다른지역을 갔을수도 있기때문에 3번 이상으로 해야하나? 그렇다기엔 3지역은 두명뿐이다....
    


In [ ]:
agg_dict = [
    ("서울",lambda x:(x.str.contains("강남|본 점|잠실|미아|영등포|분당|노원|관악|청량리")).sum()>0),
    ("경기,충청도",lambda x:(x.str.contains("인천|부평|일산|대전|안양")).sum()>0),
    ("경상도",lambda x:(x.str.contains("부산|센텀|동래|울산|창원|대구|상인|포항")).sum()>0),
    ("전라도",lambda x:(x.str.contains("광주|전주")).sum()>0)
]


tmp= df.groupby("고객번호")["지점"].agg(agg_dict).reset_index()
tmp["운송업자"] = tmp.iloc[:,1:].sum(axis=1) >= 2
features = features.merge(tmp,how="left")
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,전자제품구매 비율,잡화구매 비율,자식용품구매 비율,상세한 남자 용품 비율,상세한 여자 용품 비율,서울,"경기,충청도",경상도,전라도,운송업자
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,0.0,0.132353,0.000000,0.117647,0.058824,True,False,False,False,False
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,0.0,0.000000,0.000000,0.176471,0.058824,True,False,False,False,False
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,0.0,0.023256,0.000000,0.348837,0.046512,True,False,False,False,False
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,0.0,0.015152,0.106061,0.015152,0.106061,False,False,True,False,False
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,0.0,0.027027,0.126126,0.189189,0.081081,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,0.0,0.080863,0.002695,0.002695,0.040431,False,False,True,False,False
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,0.0,0.074380,0.000000,0.024793,0.033058,True,False,False,False,False
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.0,0.000000,0.000000,0.000000,0.333333,False,False,False,False,False
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,0.0,0.000000,0.000000,0.181818,0.181818,False,False,False,False,False


## 명절날의 제외하고 수도권과 지방을 간사람이나 지방에서 지방을 간사람을 찾아보면 그사림이 더 운송업자일 확률이 높지 않을까?
- 결과가 명절날 포함이나 제외나 똑같다
- 그래서 이건 merge안하고 위에껄로 merge함.
- 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥

- 🚛 명절 🚛
    -  🚛 2월17일, 2월 18일,2월 19일 설날
    -  🚛 9월 24일, 9월 25일 , 9월 26일 추석


In [ ]:
list_of_dates =['2007-02-17','2007-02-18','2007-02-19','2007-09-24','2007-09-25','2007-09-26']
mask_date = df["구매일자"].isin(list_of_dates) == False
tmp_date = df.loc[mask_date]
tmp_date

,고객번호,구매일자,지점,상품번호,중분류,대분류,구매가격
0,10004,2007-06-26,강남점,234160,영 트렌디,캐주얼,2720000
1,10004,2007-07-08,본 점,165162,4대 B/D,피혁잡화,8550000
2,10004,2007-07-14,본 점,663110,국산냉장고.세탁기,대형가전,28800000
3,10004,2007-07-28,본 점,73119,상품군미지정,기타,81500
4,10004,2007-07-28,본 점,73119,주류,주류,2037500
...,...,...,...,...,...,...,...
167460,49974,2007-11-07,강남점,117212,스카프(특정),섬유잡화,720000
167461,49974,2007-11-25,잠실점,39139,산지통합,농산물,161000
167462,49974,2007-05-31,강남점,85130,즉석.스넥(매장),가공식품,15000
167463,49974,2007-08-24,강남점,243201,란제리,란제리/내의,696000


In [ ]:
agg_dict = [
    ("서울",lambda x:(x.str.contains("강남|본 점|잠실|미아|영등포|분당|노원|관악|청량리")).sum()>0),
    ("경기,충청도",lambda x:(x.str.contains("인천|부평|일산|대전|안양")).sum()>0),
    ("경상도",lambda x:(x.str.contains("부산|센텀|동래|울산|창원|대구|상인|포항")).sum()>0),
    ("전라도",lambda x:(x.str.contains("광주|전주")).sum()>0)
]


loc_date = tmp_date.groupby("고객번호")["지점"].agg(agg_dict).reset_index()
loc_date["운송업자"] = loc_date.iloc[:,1:].sum(axis=1) >= 2
loc_date

,고객번호,서울,"경기,충청도",경상도,전라도,운송업자
0,10004,True,False,False,False,False
1,10013,True,False,False,False,False
2,10023,True,False,False,False,False
3,10040,False,False,True,False,False
4,10052,True,True,False,False,True
...,...,...,...,...,...,...
2548,49813,False,False,True,False,False
2549,49892,True,False,False,False,False
2550,49925,False,False,False,False,False
2551,49943,False,False,False,False,False


## 요일별 구매비율
- ⚡요일별 구매비율로 나눈 이유는 보통 남성들은 평일날 회사에 갔을 확률이 높아서
평일날 구매비율이 높은 고객들은 여성들로 예측할수 있지않을까 해서 특성 추가했습니당.
 

In [ ]:
features = features.iloc[:,:43]
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2553
Data columns (total 43 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   고객번호          2554 non-null   int64  
 1   내점일수          2554 non-null   int64  
 2   구매주기          2554 non-null   float64
 3   주말방문비율        2554 non-null   float64
 4   주구매요일         2554 non-null   int64  
 5   봄구매비율         2554 non-null   float64
 6   여름구매비율        2554 non-null   float64
 7   가을구매비율        2554 non-null   float64
 8   겨울구매비율        2554 non-null   float64
 9   일평균구매건수       2554 non-null   float64
 10  거래개월수         2554 non-null   int64  
 11  방문지점수         2554 non-null   int64  
 12  주구매지점         2554 non-null   object 
 13  상품번호수         2554 non-null   int64  
 14  중분류수          2554 non-null   int64  
 15  주구매중분류        2554 non-null   object 
 16  대분류수          2554 non-null   int64  
 17  주구매대분류        2554 non-null   object 
 18  총구매액          2554 non-null 

In [ ]:
agg_list=[
    ("월요일 구매비율",lambda x:(x.dt.weekday ==0).mean()),
    ("화요일 구매비율",lambda x:(x.dt.weekday ==1).mean()),
    ("수요일 구매비율",lambda x:(x.dt.weekday ==2).mean()),
    ("목요일 구매비율",lambda x:(x.dt.weekday ==3).mean()),
    ("금요일 구매비율",lambda x:(x.dt.weekday ==4).mean()),
    ("토요일 구매비율",lambda x:(x.dt.weekday ==5).mean()),
    ("일요일 구매비율",lambda x:(x.dt.weekday ==6).mean())
]
tmp = df.groupby("고객번호")["구매일자"].agg(agg_list,i).reset_index()
features = features.merge(tmp,how="left")
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,경상도,전라도,운송업자,월요일 구매비율,화요일 구매비율,수요일 구매비율,목요일 구매비율,금요일 구매비율,토요일 구매비율,일요일 구매비율
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,False,False,False,0.029412,0.147059,0.044118,0.058824,0.102941,0.352941,0.264706
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,False,False,False,0.000000,0.058824,0.058824,0.117647,0.647059,0.058824,0.058824
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,False,False,False,0.069767,0.023256,0.186047,0.255814,0.116279,0.093023,0.255814
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,True,False,False,0.242424,0.212121,0.090909,0.075758,0.257576,0.121212,0.000000
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,False,False,True,0.045045,0.162162,0.216216,0.126126,0.117117,0.153153,0.180180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,True,False,False,0.121294,0.218329,0.177898,0.123989,0.191375,0.035040,0.132075
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,False,False,False,0.099174,0.000000,0.214876,0.157025,0.165289,0.132231,0.231405
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,False,False,False,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,False,False,False,0.090909,0.000000,0.000000,0.090909,0.272727,0.545455,0.000000


In [ ]:
agg_list= [
    ("평일구매비율",lambda x:(x.dt.weekday < 5).mean()),
    ("주말구매비율",lambda x:(x.dt.weekday > 4).mean())

]
tmp = df.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
features = features.merge(tmp,how="left")
features

,고객번호,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,일평균구매건수,...,운송업자,월요일 구매비율,화요일 구매비율,수요일 구매비율,목요일 구매비율,금요일 구매비율,토요일 구매비율,일요일 구매비율,평일구매비율,주말구매비율
0,10004,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,2.615385,...,False,0.029412,0.147059,0.044118,0.058824,0.102941,0.352941,0.264706,0.382353,0.617647
1,10013,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,1.545455,...,False,0.000000,0.058824,0.058824,0.117647,0.647059,0.058824,0.058824,0.882353,0.117647
2,10023,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,2.047619,...,False,0.069767,0.023256,0.186047,0.255814,0.116279,0.093023,0.255814,0.651163,0.348837
3,10040,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,2.750000,...,False,0.242424,0.212121,0.090909,0.075758,0.257576,0.121212,0.000000,0.878788,0.121212
4,10052,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,3.964286,...,True,0.045045,0.162162,0.216216,0.126126,0.117117,0.153153,0.180180,0.666667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,4.469880,...,False,0.121294,0.218329,0.177898,0.123989,0.191375,0.035040,0.132075,0.832884,0.167116
2550,49892,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,3.903226,...,False,0.099174,0.000000,0.214876,0.157025,0.165289,0.132231,0.231405,0.636364,0.363636
2551,49925,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,1.000000,...,False,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.666667,0.333333
2552,49943,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,1.571429,...,False,0.090909,0.000000,0.000000,0.090909,0.272727,0.545455,0.000000,0.454545,0.545455


In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2553
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   고객번호          2554 non-null   int64  
 1   내점일수          2554 non-null   int64  
 2   구매주기          2554 non-null   float64
 3   주말방문비율        2554 non-null   float64
 4   주구매요일         2554 non-null   int64  
 5   봄구매비율         2554 non-null   float64
 6   여름구매비율        2554 non-null   float64
 7   가을구매비율        2554 non-null   float64
 8   겨울구매비율        2554 non-null   float64
 9   일평균구매건수       2554 non-null   float64
 10  거래개월수         2554 non-null   int64  
 11  방문지점수         2554 non-null   int64  
 12  주구매지점         2554 non-null   object 
 13  상품번호수         2554 non-null   int64  
 14  중분류수          2554 non-null   int64  
 15  주구매중분류        2554 non-null   object 
 16  대분류수          2554 non-null   int64  
 17  주구매대분류        2554 non-null   object 
 18  총구매액          2554 non-null 